<a href="https://colab.research.google.com/github/andreahorbach/SeminarMSV/blob/main/TR_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sitzung 5: Plagiatserkennung - Teil 2
Heute schauen wir uns an, wie man mit Ähnlichkeitsmethoden einen Plagiatserkenner trainiert. Wir bauen auf die Ähnlichkeitsmaße aus der letzten Sitzung auf.

**Schritt 1: String Similarity / Oberflächenähnlichkeit**
Dieser Code sollte Ihnen aus der letzten Sitzung bekannt vorkommen.

In [ ]:
!pip install -U strsimpy # installiere die notwendigen Pakete
!pip install pylcs

In [ ]:
from strsimpy.levenshtein import Levenshtein
from pylcs import lcs_string_length

levenshtein = Levenshtein()

# Beispiele, die Sie zum Austesten verändern können
s1 = 'Ein Beispiel'
s2 = 'Ein Beispöl'
s3 = 'Anna isst einen Apfel'
s4 = 'Anna ist eine Apfelsine.'

#Levenshtein distance
print(levenshtein.distance(s1, s2))
print(levenshtein.distance(s3, s4))

# Longest Common Substring
print(lcs_string_length(s1, s2))
print(lcs_string_length(s3, s4))


**Schritt 2: Semantic Similarity**
Auch diesen Code kennen Sie bereits.

In [18]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [4]:
def compute_semantic_similarity(s1, s2):
    embeddings1 = model.encode([s1], convert_to_tensor=True)
    embeddings2 = model.encode([s2], convert_to_tensor=True)
    return util.cos_sim(embeddings1, embeddings2)[0][0]

print("{:.4f}".format(compute_semantic_similarity("Mary walks home.", "Mary goes to her house")))
print("{:.4f}".format(compute_semantic_similarity("Maria geht nach Hause.", "Maria fährt heim.")))

0.8669
0.9184


**Aufgabe 1 (von letzter Woche):** Nehmen Sie an, Sie möchten den ersten Abschnitt des Wikipediaartikels zum Thema Plagiat plagiieren. Er lautet "Ein Plagiat ist die Anmaßung fremder geistiger Leistungen. Dies kann sich auf die Übernahme fremder Texte oder anderer Darstellungen bzw. Ideen beziehen, z. B. Zeitungs-, Magazinartikel, Fotos, Filme, Tonaufnahmen, Musik sowie Erfindungen, Design, wissenschaftliche Erkenntnisse, Melodien oder beides gleichzeitig."

Probieren Sie, wie Sie diese beiden Sätze so umformen können, dass durch Ihre Methoden möglichst nicht mehr direkt als Plagiat erkennbar sind. Probieren Sie verschiedene Optionen aus und beobachten Sie, wie sich die Werte der Metriken verändern.

**Schritt 3:** "To be or not to be that is the question" schrieb Shakespeare. Finden Sie - für jedes der drei Maße - den ähnlichsten Satz bei Jane Austen. Ist der Satz ein Plagiat? Hier eine mögliche Lösung - ebenfalls von letzter Woche.

In [ ]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
from nltk.corpus import gutenberg

austen1 = gutenberg.sents('austen-emma.txt')[0:500] # consider only the first 500 sentence, can be adjusted depending on how long you want to wait
shakespeare = "To be or not to be that is the question ."
maxSim = 0.0
bestSentence = "dummy"
for sent in austen1:
  text = " ".join(sent) # macht aus der Liste von Tokens wieder einen Text.
  sim = compute_semantic_similarity(shakespeare, text)
  if (sim > maxSim):
      print("new most similar sentence: ", sim, text)
      maxSim = sim
      bestSentence = text
print("Final results: ", maxSim, bestSentence)


**Schritt 4:** Um nach "echten " Plagiaten suchen können, ergänzen wir das Jane Austen Corpus um einige plagiierte Versionen des Shakespear-Zitates. Außerdem merken wir uns in einem separaten Array für jeden Satz das Label als kein-Plagiat (K) oder Plagiat (P). (Sie erinnern sich, das haben wir beim Machine Learninig vor zwei Sitzungen auch schon so gemacht) Natürlich sind nur unsere Ergänzungen Plagiate.

In [ ]:
plagiate = ["Not to be or to be is an important question", "I am wondering whether one should be.", "Existence is very relevant", "Sein oder nichtsein, das ist hier die Frage"]
sentences = plagiate + [" ".join(sent) for sent in austen1]
gold_labels = len(plagiate)*["P"] + ["K" for sent in austen1]
print(sentences)
print(gold_labels)

**Aufgabe 2:** Ergänzen Sie 5 weitere Sätze, die Plagiate des Shakespearezitats sind mit dem Label P.

**Aufgabe 3:** Benutzen Sie den Algorithmus von oben und suchen Sie  Satz aus 'sentence', der am ähnlichsten zum Shakespearezitat ist. Ist er ein Plagiat?

**Aufgabe 4:** Als nächstes sollen Sie die Sätze nach ihrer Ähnlichkeit zum Originalzitat sortieren. Sie müssen sich dafür also jede Ähnlichkeit zusammen mit dem Satz merken. Legen Sie dazu ein neues Array an, in dem Sie für jeden Satz die Similarity zum Shakespearezitat speichern.

Achtung: Sie haben dann am Ende drei Arrays, eines mit den Sätzen, eines mit den Gold-Labels und eines mit den Similarities. Sie wollen alle drei Essays auf die gleiche Art sortieren, nämlich absteigend nach dem Wert des Similarity-Arrays. Hier ist Beispielcode für die Sortierung, den Sie für die Lösung der Aufgabe verwenden können.

In [ ]:
# Example arrays
array1 = [3, 1, 4, 1, 5, 9, 2, 6]
array2 = ['C', 'A', 'D', 'A', 'E', 'I', 'B', 'F']
array3 = ['apple', 'banana', 'cherry', 'avocado', 'elderberry', 'kiwi', 'blueberry', 'fig']

# Sort all three arrays based on the values of array1
sorted_arrays = sorted(zip(array1, array2, array3), key=lambda x: x[0])

# Unpack the sorted tuples back into separate arrays
sorted_array1, sorted_array2, sorted_array3 = zip(*sorted_arrays)

# Print the sorted arrays
print("Sorted Array 1:", sorted_array1)
print("Sorted Array 2:", sorted_array2)
print("Sorted Array 3:", sorted_array3)

**Aufgabe 5:** Setzen Sie einen Threshold bei 0.9, so dass sie jeden Text mit einem Similarity-Wert über 0.9 als Plagiat behandeln und berechnen Sie Precision und Recall.

Auch hier gibt es in der nächsten Codezelle ein Beispiel, wie Sie Precision, Recall und F-Score mit Hilfe des sklearn packages berechnen. Überlegen Sie , wie Sie die true labels und die predicted labels für Ihr Beispiel befüllen.

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Example true labels and predicted labels
true_labels = [1, 0, 1, 1, 0, 1, 0, 1]
predicted_labels = [1, 0, 1, 0, 0, 1, 0, 0]

# Specify the positive class (class '1')
positive_class = 1

# Compute precision for class '1'
precision = precision_score(true_labels, predicted_labels, pos_label=positive_class)

# Compute recall for class '1'
recall = recall_score(true_labels, predicted_labels, pos_label=positive_class)

# Compute F1 score for class '1'
f1 = f1_score(true_labels, predicted_labels, pos_label=positive_class)

# Print the results
print("Precision for class '1': {:.2f}".format(precision))
print("Recall for class '1': {:.2f}".format(recall))
print("F1 Score for class '1': {:.2f}".format(f1))

**Aufgabe 6:**  Finden Sie durch ausprobieren den Threshold, der den F-Score maximiert.